In [2]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)
import pandas as pd

In [7]:
engine = create_engine('postgresql+psycopg2://user@localhost/llm')
df_diabeties  = pd.read_csv('data/diabetes.csv')
df_diabeties.to_sql('diabetes', con = engine, if_exists='replace', index = False)

df_customers  = pd.read_csv('data/customers.csv')
df_customers.to_sql('customers', con = engine, if_exists='replace', index = False)

df_flights  = pd.read_csv('data/flights.csv')
df_flights.to_sql('flights', con = engine, if_exists='replace', index = False)

df_transactions  = pd.read_csv('data/transactions.csv')
df_transactions.to_sql('transactions', con = engine, if_exists='replace', index = False)


16

In [67]:
from langchain.agents import tool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.utilities.sql_database import SQLDatabase
@tool
def db_list_tables(db_name:str):
    """Input is database name, output is a comma-separated list of tables in the database."""
    db = SQLDatabase.from_uri(f'postgresql+psycopg2://user@localhost/{db_name}')
    return ', '.join(db.get_usable_table_names())

class db_schema_input(BaseModel):
    table_names: str = Field(
        ...,
        description=(
            "A comma-seperated list of the table names for which to return the schema. "
            "Example input: 'table1, table2, table3' "
        ),
    )
    db_name:str = Field(description="Name of the database")
    
@tool(args_schema= db_schema_input)
def db_schema(table_names:str, db_name:str):
    """Get the schema and sample rows for the specified SQL tables."""
    db = SQLDatabase.from_uri(f'postgresql+psycopg2://user@localhost/{db_name}') 
    return db.get_table_info_no_throw(
        [t.strip() for t in table_names.split(",")]
    )


from langchain.agents import tool
from langchain_core.pydantic_v1 import BaseModel, Field

# Tool to list all the table names available in the database
@tool
def db_list_tables(db_name:str):
    """Input is database name, output is a comma-separated list of tables in the database."""
    db = SQLDatabase.from_uri(f"postgresql+psycopg2://user@localhost/{db_name}")
    return ", ".join(db.get_usable_table_names())

# Too to get the schema and first few rows of all the tables in the database
## For adding description to the argument of the function we will use pydantic
class db_schema_input(BaseModel):
    table_names: str = Field(
        ...,
        description=(
            "A comma-separated list of the table names for which to return the schema. "
            "Example input: 'table1, table2, table3'"
        ),
    )
    db_name:str = Field(description="Name of the database")

@tool(args_schema=db_schema_input) # Define the input schema
def db_schema(table_names:str,db_name:str):
    """Get the schema and sample rows for the specified SQL tables."""
    db = SQLDatabase.from_uri(f"postgresql+psycopg2://user@localhost/{db_name}")
    return db.get_table_info_no_throw(
            [t.strip() for t in table_names.split(",")]
        )

tools = [db_schema,db_list_tables]

In [66]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.utilities.sql_database import SQLDatabase

class db_schema_input(BaseModel):
    table_names: str = Field(
        ...,
        description=(
            "A comma-separated list of the table names for which to return the schema. "
            "Example input: 'table1, table2, table3' "
        ),
    )
    db_name: str = Field(description="Name of the database")

@tool(args_schema=db_schema_input)
def db_schema(table_names: str, db_name: str):
    """Get the schema and sample rows for the specified SQL tables."""
    try:
        print(table_names)
        db = SQLDatabase.from_uri(f'postgresql+psycopg2://user@localhost/{db_name}')
        
        return
        table_list = [t.strip() for t in table_names.split(",")]
        print(table_list)
        
        print(f"Retrieving schema for tables: {table_list}")  # Debugging output
        result = db.get_table_info_no_throw(table_list)
        return result
    except AttributeError as e:
        print(f"AttributeError: {e}")
        raise
    except Exception as e:
        print(f"Unexpected error: {e}")
        raise
print(db_list_tables('llm'))
db_instance = db_schema_input(table_names=db_list_tables('llm'),db_name='llm')
# # list of tools
# db_schema(db_instance.table_names, db_instance.db_name)
db_instance.db_schema()


city_stats, customers, diabetes, flights, transactions


AttributeError: 'db_schema_input' object has no attribute 'db_schema'

In [33]:
from langchain_core.messages import AIMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_community.agent_toolkits.sql.prompt import (
    SQL_FUNCTIONS_SUFFIX,
    SQL_PREFIX,
    SQL_SUFFIX,
)

# Prompt
system_message = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct mysql query to run.
You have access to tools for creating the query.
Only use the below tools. Only use the information returned by the below tools to construct your query.

If the question does not seem related to the database, just return "I don't know" as the answer.

"""

ai_message = """
'I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
"""

messages = [
    
    SystemMessage(content=system_message),
    HumanMessagePromptTemplate.from_template("{input}"),
    AIMessage(content=ai_message),
    MessagesPlaceholder(variable_name="agent_scratchpad")
]

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(messages)

In [88]:
from langchain_community.llms import Ollama
llm = Ollama(model='llama3')
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv
import os
# load_dotenv()
# api_key = os.getenv("ANTHROPIC_API_KEY")
# llm = ChatAnthropic(model="claude-2", api_key= api_key, temperature = 0)

In [ ]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

model = OllamaFunctions(model="llama3")

model = model.bind(
    functions=[
        {
            "name": "route",
            "description": "Select the next role.",
            "parameters": {
                "title": "routeSchema",
                "type": "object",
                "properties": {"next": {"title": "Next", "anyOf": [{"enum": options}] }},
                "required": ["next"],
            },
        }
    ],
    function_call={"name": "route"},
)

In [97]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

from groq import Groq
llm = Groq(
    api_key = api_key
)

In [77]:
from langchain_core.runnables.passthrough import RunnablePassthrough
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain.agents.output_parsers.tools import ToolsAgentOutputParser
def create_tool_calling_agent(
    llm, tools, prompt
):
#-> Runnable:
    """Create an agent that uses tools.

    Args:
        llm: LLM to use as the agent.
        tools: Tools this agent has access to.
        prompt: The prompt to use. See Prompt section below for more on the expected
            input variables.

    Returns:
        A Runnable sequence representing an agent. It takes as input all the same input
        variables as the prompt passed in does. It returns as output either an
        AgentAction or AgentFinish.

    Example:

        .. code-block:: python

            from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
            from langchain_anthropic import ChatAnthropic
            from langchain_core.prompts import ChatPromptTemplate

            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", "You are a helpful assistant"),
                    ("placeholder", "{chat_history}"),
                    ("human", "{input}"),
                    ("placeholder", "{agent_scratchpad}"),
                ]
            )
            model = ChatAnthropic(model="claude-3-opus-20240229")

            @tool
            def magic_function(input: int) -> int:
                \"\"\"Applies a magic function to an input.\"\"\"
                return input + 2

            tools = [magic_function]

            agent = create_tool_calling_agent(model, tools, prompt)
            agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

            agent_executor.invoke({"input": "what is the value of magic_function(3)?"})

            # Using with chat history
            from langchain_core.messages import AIMessage, HumanMessage
            agent_executor.invoke(
                {
                    "input": "what's my name?",
                    "chat_history": [
                        HumanMessage(content="hi! my name is bob"),
                        AIMessage(content="Hello Bob! How can I assist you today?"),
                    ],
                }
            )

    Prompt:

        The agent prompt must have an `agent_scratchpad` key that is a
            ``MessagesPlaceholder``. Intermediate agent actions and tool output
            messages will be passed in here.
    """
    missing_vars = {"agent_scratchpad"}.difference(
        prompt.input_variables + list(prompt.partial_variables)
    )
    #Exception handling
    if missing_vars:
        raise ValueError(f"Prompt missing required variables: {missing_vars}")

    if not hasattr(llm, "bind_tools"):
        raise ValueError(
            "This function requires a .bind_tools method be implemented on the LLM.",
        )
    # Binding tools to llm  
    # functions = [format_tool_to_openai_function(t) for t in tools]
    llm_with_tools = llm.bind_functions(tools)
    # Agent chain
    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_to_tool_messages(x["intermediate_steps"])
        )
        | prompt
        | llm_with_tools
        | ToolsAgentOutputParser()
    )
    return agent

In [89]:
from langchain.agents import AgentExecutor
from langchain.agents import AgentExecutor, create_tool_calling_agent
sql_agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=sql_agent, tools=tools, verbose=True)

In [91]:
agent_executor.invoke({"input": "how are you"})



> Entering new AgentExecutor chain...


BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}

In [90]:
agent_executor.invoke({"input": "Can you tell me name of top 3 customers who booked the most flights in flight_reservations databse"})



> Entering new AgentExecutor chain...


BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}

**vanna**

In [98]:
#importing Ollama, which allows users to connect with any open-source LLM
from vanna.ollama import Ollama
from vanna.vannadb import VannaDB_VectorStore

class MyVanna(VannaDB_VectorStore, Ollama):
    def __init__(self, config=None):
        MY_VANNA_MODEL = # Your model name from https://vanna.ai/account/profile
        VannaDB_VectorStore.__init__(self, vanna_model=MY_VANNA_MODEL, vanna_api_key=MY_VANNA_API_KEY, config=config)
        Ollama.__init__(self, config=config)

# use llama3:70B for the 70B model
vn = MyVanna(config={'model': 'llama3'})

SyntaxError: invalid syntax (3771830969.py, line 7)